In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv("./lab3_data/reviews.tsv", sep="\t")
df

,sentiment,productId,userId,summary,text,helpfulY,helpfulN
0,-1,B001E5E1KA,A3BPS5Z69S82MV,Dissappointed,"I bought this product, because it's slightly l...",2,1
1,-1,B001CPQPXI,A1WCT6YAYIWLRV,Hard as rocks,Unless your dog is younger and has great teeth...,0,0
2,-1,B00654I0XE,A45NFCAENHRMD,Some products contained gluten.,I wouldn't recommend this to a celiac buyer. A...,0,1
3,-1,B000FBM480,A1FEP94TP15RG0,"These ""Butter Leaves"" are addictively wonderful!",Amazon sends you TWELVE boxes of these fabulou...,0,11
4,-1,B002KB2CMG,A31CQ4G2QFVMDE,Oregon versus Turkey,I bought a bag of the Oregon hazelnuts and a b...,0,0
...,...,...,...,...,...,...,...
9995,-1,B001UOJGNM,A1SXUCZYHQVMY,Pretty good,My bottle arrived early and I was very excited...,1,0
9996,-1,B003QU2LYY,A2EHF9TF8XG4UI,Just Okay,Nothing spectacular. It wasn't as sweet as I'...,0,0
9997,1,B000N8N45W,A1D63A1P3SVYTU,"Convenient, Healthy Meal Option",This is perfect for the office. No refrigerat...,2,0
9998,1,B001EO7NJ2,A30MN8FL7ZS1UF,DELICIOUS !,"This licorice is sooooo good. Soft, very flavo...",2,0


In [31]:
a = df["summary"].str.lower().str.split()
dict = {}
cur_idx = 0
for i in range(len(a)):
    for j in a[i]:
        if j not in dict:
            dict[j] = cur_idx
            cur_idx += 1
    

In [32]:
data = np.zeros((cur_idx, df.shape[0]))
labels = np.array([df["sentiment"].to_numpy()])
d, n = data.shape

for i in range(len(a)):
    for j in a[i]:
        data[dict[j], i] = 1

In [33]:
def averaged_perceptron(data, labels, T):
    d, n = data.shape
    theta = np.zeros((d,1))
    theta_0 = np.zeros(1)
    ths = np.zeros((d,1))
    th0s = np.zeros(1)
  
    for t in range(T):     
      for i in range(n):
        y = labels[0,i]
        x = data[:,i]
        
        a = np.dot(x,theta)+theta_0
        if np.sign(y*a)[0] <=0: # update the thetas
          theta[:,0] = theta[:,0]+ y*x
          theta_0 = theta_0 + y
        
        ths += theta
        th0s += theta_0
    return (ths/(n*T),th0s/(n*T))

def score(data_test, labels_test, th, th0):
    d, n = data_test.shape
    ret = 0
    for i in range(n):
        x = data_test[:, i]
        y = labels_test[0, i]
        a = np.dot(x, th)+th0
        if np.sign(a)[0]==y:
            ret+=1
    return ret

def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    th, th0 = learner(data_train, labels_train, 100)
    return score(data_test, labels_test, th, th0)/labels_test.shape[1]

def xval_learning_alg(learner, data, labels, k):
  d, n = data.shape
  a = np.array_split(data, k, axis=1)
  b = np.array_split(labels, k, axis=1)
  score = 0
  for i in range(k):
    # initialize testing data/labels
    data_test = a[i]
    label_test = b[i]

    data_train = np.zeros((d, 1))
    label_train = np.zeros((1, 1))
    for j in range(0, k):
      if i==j:
        continue
      data_train = np.concatenate((data_train, a[j]), axis=1)
      label_train = np.concatenate((label_train, b[j]), axis=1)
    data_train = np.delete(data_train, 0, 1)
    label_train = np.delete(label_train, 0, 1)
    score += eval_classifier(learner, data_train, label_train, data_test, label_test)
  return score/k



In [34]:
xval_learning_alg(averaged_perceptron, data, labels, 10)

0.7837000000000001